# US airline sentiment analysis
data https://www.kaggle.com/crowdflower/twitter-airline-sentiment

the scope of this projecy to classify the tweets from US airlinr users to detemined wither it's +ve,-ne or neutral

avilable data as CSV file contain tweet_id, airline_sentiment, name, text, tweet_coord, tweet_created, tweet_location, user_timezone

the most importent airline_sentiment , text(feature)
### steps
#### read data
#### data cleaning (punchtuation removal , case lowering , stop words removel )
#### create bag of words
#### select model / train
#### Predicte and Evaluate the Model


## data reading
reading data from CSV using Pandas

download CSV file to the same directory

In [2]:
import pandas as pd

file_path = './Tweets.csv' 
tweets_data=pd.read_csv(file_path)

tweets_data.head()

,tweet_id,airline_sentiment,name,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,5.703060e+17,neutral,cairdin,@VirginAmerica What @dhepburn said.,NaN,2/24/2015 11:35,NaN,Eastern Time (US & Canada)
1,5.703010e+17,positive,jnardino,@VirginAmerica plus you've added commercials t...,NaN,2/24/2015 11:15,NaN,Pacific Time (US & Canada)
2,5.703010e+17,neutral,yvonnalynn,@VirginAmerica I didn't today... Must mean I n...,NaN,2/24/2015 11:15,Lets Play,Central Time (US & Canada)
3,5.703010e+17,negative,jnardino,@VirginAmerica it's really aggressive to blast...,NaN,2/24/2015 11:15,NaN,Pacific Time (US & Canada)
4,5.703010e+17,negative,jnardino,@VirginAmerica and it's a really big bad thing...,NaN,2/24/2015 11:14,NaN,Pacific Time (US & Canada)


### Cleaning Process
remove punc.,special characters,tags and mentions,links, extra spaces

lowering all letters 

In [3]:
import string
import re

def clean_steps(text):
    text = re.sub (r'@[A-za-z0-9]+','',text) #remve tags and mentions 
    text = re.sub (r'https?://[A-za-z0-9]+','',text) # remove any 
    
    text = re.sub('[‘’“”…]', ' ', text)
    text = text.lower()
    text = re.sub('\[*.?\@]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)   
    text = re.sub('\n', '', text)
    
    return text

clean_process = lambda x: clean_steps(x)

In [4]:
tweets_txt = tweets_data.drop(['tweet_id','name','tweet_coord','tweet_created','tweet_location','user_timezone'],axis=1)

tweets_txt_cleaned = pd.DataFrame(tweets_txt.text.apply(clean_process))
tweets_txt_cleaned.columns = ['cleaned_punch']
tweets_txt_cleaned.head()

,cleaned_punch
0,what said
1,plus youve added commercials to the experienc...
2,i didnt today must mean i need to take anothe...
3,its really aggressive to blast obnoxious ente...
4,and its a really big bad thing about it


## remove redundent words
Stop Words he,she,it,there.this....etc

stemming words to reduce no. of words that give the same meading (added -> add)


In [5]:
import nltk 

def stemmin_steps(text):
    
    stemmer = nltk.PorterStemmer()
    
    word_tokens = nltk.tokenize.word_tokenize(text)
    
    out_text = [stemmer.stem(word) for word in word_tokens]      
    return out_text

stemmin_process = lambda x: stemmin_steps(x)


In [6]:
tweets_stem = pd.DataFrame(tweets_txt_cleaned.cleaned_punch.apply(stemmin_process))
tweets_stem.columns = ['stemmed_token']
tweets_stem.head()

,stemmed_token
0,"[what, said]"
1,"[plu, youv, ad, commerci, to, the, experi, tacki]"
2,"[i, didnt, today, must, mean, i, need, to, tak..."
3,"[it, realli, aggress, to, blast, obnoxi, enter..."
4,"[and, it, a, realli, big, bad, thing, about, it]"


In [7]:
def remove_stop_words(text_token):
    
    stop_words = set(nltk.corpus.stopwords.words('english')) 
        
    out_text = [w for w in text_token if not w in stop_words]
  
    return out_text

removing_stop_words = lambda x: remove_stop_words(x)

In [8]:
tweets_filtered = pd.DataFrame(tweets_stem.stemmed_token.apply(removing_stop_words))
tweets_filtered.columns = ['filtered_token']
tweets_filtered.head()

,filtered_token
0,[said]
1,"[plu, youv, ad, commerci, experi, tacki]"
2,"[didnt, today, must, mean, need, take, anoth, ..."
3,"[realli, aggress, blast, obnoxi, entertain, gu..."
4,"[realli, big, bad, thing]"


### Bag of words / feature Vector
use the most freq. words as a bag of word then create feacture vector using this words 
get token and check unique wods after stemming 


In [9]:
import itertools

def uniqe_words(coloum):
    return len(set (list(itertools.chain.from_iterable(coloum))))

tweets_stem_len        = uniqe_words(tweets_stem['stemmed_token'])
tweets_filtered_len    = uniqe_words(tweets_filtered['filtered_token'])

print ('tweets_stem_len = '+str(tweets_stem_len))
print ('tweets_filtered_len = '+str(tweets_filtered_len))

tweets_stem_len = 10132
tweets_filtered_len = 10014


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
def dummy_vec(txt):
    return txt
vectorizer = CountVectorizer( tokenizer=dummy_vec ,preprocessor=dummy_vec,max_features=2500, min_df=10, max_df=0.8)

feature_vectore= vectorizer.fit_transform(tweets_filtered['filtered_token']).toarray() 

change the out put value from string to numbers
maybe used later on


In [11]:
def enum_out(text):
    num =0
    if text == "negative":
        num =-1
    elif text == "positive":
        num = 1
    return num
enuming_out = lambda x:enum_out(x)
y_val = pd.DataFrame(tweets_txt.airline_sentiment.apply(enuming_out))

splitting the data to Test & train (70%-30%) 

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature_vectore,y_val,test_size=0.3,random_state=101)

### create the Classifier using Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)  
text_classifier.fit(X_train, y_train.values.ravel())  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

create func to view the acc. of the model


In [15]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
def get_pred(x_test,y_test,model):
    predictions = model.predict(X_test) 
    print(confusion_matrix(y_test,predictions))  
    print(classification_report(y_test,predictions))  
    print(accuracy_score(y_test, predictions))  

In [16]:
get_pred(X_test,y_test,text_classifier)

[[2451  194   93]
 [ 407  438  102]
 [ 210   75  422]]
             precision    recall  f1-score   support

         -1       0.80      0.90      0.84      2738
          0       0.62      0.46      0.53       947
          1       0.68      0.60      0.64       707

avg / total       0.74      0.75      0.74      4392

0.7538706739526412
